In [ ]:
...
@app.get('/programming_languages')
def list_programming_languages():
   before_year = request.args.get('before_year') or '30000'
   after_year = request.args.get('after_year') or '0'
   qualifying_data = list(
       filter(
           lambda pl: int(before_year) > pl['publication_year'] > int(after_year),
           in_memory_datastore.values()
       )
   )

   return {"programming_languages": qualifying_data}

In [ ]:
data_opioid.columns

In [ ]:
unique_data.columns

In [ ]:
data_opioid = data_opioid.rename({'OUD_Score': 'Opioid_Score', 'PatientId': 'patient_id'}, axis=1)
unique_data = unique_data.rename({'OUD_Score': 'Opioid_Score', 'PatientId': 'patient_id'}, axis=1)

In [ ]:
data_opioid.fillna("NA", inplace=True)
unique_data.fillna("NA", inplace=True)

In [ ]:
def column_format(data):
    data['Opioid_Risk'] = 'Medium'
    data['Random_Forest_Probability'] = data['Random_Forest_Probability'].apply(lambda x: '{:.5f}'.format(x))
    data['Calibrated_Random_Forest_Probability'] = data['Calibrated_Random_Forest_Probability'].apply(
        lambda x: '{:.5f}'.format(x))
    data['Naive_Bias_Probability'] = data['Naive_Bias_Probability'].apply(lambda x: '{:.5f}'.format(x))
    data['Isotonic_Calibrated_Naive_Bias_Probability'] = data['Isotonic_Calibrated_Naive_Bias_Probability'].apply(
        lambda x: '{:.5f}'.format(x))
    data['Sigmoid_Calibrated_Naive_Bias_Probability'] = data['Sigmoid_Calibrated_Naive_Bias_Probability'].apply(
        lambda x: '{:.5f}'.format(x))
    data['Opioid_Score'] = data['Opioid_Score'].apply(lambda x: '{:.5f}'.format(x))

    data.loc[data['Opioid_Score'].astype(float) > 0.7, 'Opioid_Risk'] = 'High'
    data.loc[data['Opioid_Score'].astype(float) < 0.5, 'Opioid_Risk'] = 'Low'
    data = data.reindex(columns=['patient_id', 'Opioid_Risk', 'Opioid_Score',
                                 'Calibrated_Random_Forest_Probability', 'Isotonic_Calibrated_Naive_Bias_Probability',
                                 'Naive_Bias_Probability',
                                 'Random_Forest_Probability', 'Sigmoid_Calibrated_Naive_Bias_Probability'])

    return data

In [ ]:
data_opioid = column_format(data_opioid)
unique_data_opioid = column_format(unique_data)


In [ ]:
data_opioid.shape, unique_data_opioid.shape

In [ ]:
data_opioid.patient_id.unique()

In [ ]:
df = data_opioid.to_json(orient="records")
df = json.loads(df)

In [ ]:
df_unique = unique_data_opioid.to_json(orient="records")
df_unique = json.loads(df_unique)

In [ ]:
len(df_unique), len(df)

In [ ]:
results = []
# Loop through the data and match results that fit the requested ID.

for id_ in range(len(df)):
    if df[id_]['patient_id'] == '4AB43785-5286-48A3-AD42-00815ED93337':
        results.append(df[id_])

# Use the jsonify function from Flask to convert our list of
# Python dictionaries to the JSON format.
if len(results) < 1:
    return "PatientId is not found", 404
else:
    return jsonify(results)

In [ ]:
df_unique['patient_id' == '4AB43785-5286-48A3-AD42-00815ED93337']

In [ ]:
df['patient_id' == '4AB43785-5286-48A3-AD42-00815ED93337']